###### 

In [1]:
import os
import yaml
import time
import shutil
import torch
import random
import argparse
import numpy as np
import torch.nn as nn
import onnx
from torch.autograd import Variable

from torch.utils import data
from ptsemseg.models import get_model
from ptsemseg.utils import convert_state_dict

In [2]:
def init_model(model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    n_classes = 19

    # Setup Model
    model = get_model({"arch": "hardnet"}, n_classes)
    state = convert_state_dict(torch.load(model_path, map_location=device)["model_state"])
    model.load_state_dict(state)
    model.eval()
    model.to(device)

    return device, model

In [3]:
model_path = 'weights/hardnet70_cityscapes_model.pkl'
device, model = init_model(model_path)

In [4]:
dummy_input = np.random.rand(512,1024,3)  # uint8 with RGB mode (h,w,c)
img = dummy_input.astype(np.float16)

In [5]:
# norm
value_scale = 255
mean = [0.406, 0.456, 0.485]
mean = [item * value_scale for item in mean]
std = [0.225, 0.224, 0.229]
std = [item * value_scale for item in std]
img = (img - mean) / std

# NHWC -> NCHW (n=batch size(한 배치 안의 이미지 개수),c,h,w)로 바꿔줌
img = img.transpose(2, 0, 1)
img = np.expand_dims(img, 0) #batch로 만들어줌
img = torch.from_numpy(img).float()
images = img.to(device) #gpu에 image올림
outputs = model(images)


In [9]:
# Export the trained model to ONNX

torch.onnx.export(model,images,"FCHarDNet.onnx",opset_version=9,verbose = True)
#torch.onnx.export(model,images,"FCHarDNet1.onnx",opset_version=9)

graph(%input.1 : Float(1:3, 3:1, 512:3072, 1024:3, requires_grad=0, device=cuda:0),
      %finalConv.weight : Float(19:48, 48:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %finalConv.bias : Float(19:1, requires_grad=1, device=cuda:0),
      %735 : Float(16:27, 3:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %736 : Float(16:1, requires_grad=0, device=cuda:0),
      %738 : Float(24:144, 16:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %739 : Float(24:1, requires_grad=0, device=cuda:0),
      %741 : Float(32:216, 24:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %742 : Float(32:1, requires_grad=0, device=cuda:0),
      %744 : Float(48:288, 32:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %745 : Float(48:1, requires_grad=0, device=cuda:0),
      %747 : Float(10:432, 48:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %748 : Float(10:1, requires_grad=0, device=cuda:0),
      %750 : Float(18:522, 58:9, 3:3, 3:1, requires_grad=0, device=cuda:0),
      %751 : Fl

RuntimeError: ONNX export failed: Couldn't export operator aten::upsample_bilinear2d

Defined at:
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/nn/functional.py(3151): interpolate
/home/dongmin/FCHarDNet_Nextchip/ptsemseg/models/hardnet.py(247): forward
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/nn/modules/module.py(709): _slow_forward
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/nn/modules/module.py(725): _call_impl
/home/dongmin/FCHarDNet_Nextchip/ptsemseg/models/hardnet.py(348): forward
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/nn/modules/module.py(709): _slow_forward
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/nn/modules/module.py(725): _call_impl
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/jit/_trace.py(116): wrapper
/home/dongmin/mindung/mindungmindung/lib/python3.6/site-packages/torch/jit/_trace.py(130): forward
/home/d

In [3]:
oml = onnx.load('FCHarDNet.onnx')  # onnx model
onnx.checker.check_model(oml)
    

SyntaxError: invalid syntax (<ipython-input-3-9f89f0481fb2>, line 1)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import yaml
import time
import shutil
import torch
import random
import argparse
import numpy as np
import torch.nn as nn
import onnx

from torch.utils import data
from ptsemseg.models import get_model
from ptsemseg.utils import convert_state_dict


# In[2]:


def init_model(model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    n_classes = 19

    # Setup Model
    model = get_model({"arch": "hardnet"}, n_classes)
    state = convert_state_dict(torch.load(model_path, map_location=device)["model_state"])
    model.load_state_dict(state)
    model.eval()
    model.to(device)

    return device, model


# In[3]:


model_path = 'weights/hardnet70_cityscapes_model.pkl'
device, model = init_model(model_path)


# In[4]:


dummy_input = np.random.rand(512,1024,3)  # uint8 with RGB mode (h,w,c)
img = dummy_input.astype(np.float16)

# norm
value_scale = 255
mean = [0.406, 0.456, 0.485]
mean = [item * value_scale for item in mean]
std = [0.225, 0.224, 0.229]
std = [item * value_scale for item in std]
img = (img - mean) / std

# NHWC -> NCHW (n=batch size(한 배치 안의 이미지 개수),c,h,w)로 바꿔줌
img = img.transpose(2, 0, 1)
img = np.expand_dims(img, 0) #batch로 만들어줌
img = torch.from_numpy(img).float()

images = img.to(device) #gpu에 image올림
outputs = model(images)


# In[5]:


torch.onnx.export(model,images,"FCHarDNet.onnx",opset_version=11,verbose = True)
#torch.onnx.export(model,images,"FCHarDNet1.onnx",opset_version=9)


# In[6]:


#print(model)

oml = onnx.load('FCHarDNet.onnx')  # onnx model
onnx.checker.check_model(oml)


# In[ ]:


In [ ]:

#cis true
label_idx_0 = []
for i in range(len(cis_test_y)):
    if(cis_test_y[i]==0):
        label_idx_0.append(i)
        
data_0 = np.zeros(shape = (len(label_idx_0),2))
for i in range(len(label_idx_0)):
    data_0[i] = cis_test_x[label_idx_0[i]]

label_idx_1 = []
for i in range(len(cis_test_y)):
    if(cis_test_y[i]==1):
        label_idx_1.append(i)
        
data_1 = np.zeros(shape = (len(label_idx_1),2))
for i in range(len(label_idx_1)):
    data_1[i] = cis_test_x[label_idx_1[i]]
    

plt.scatter(data_0[:,0], data_0[:,1], c = 'k', label='0')
plt.scatter(data_1[:,0], data_1[:,1], c = 'w', label='1')
plt.legend(loc=4)
plt.show()